In [1]:
!pip install beautifulsoup4

In [8]:
import requests
from bs4 import BeautifulSoup

# AI Hub 페이지 URL
url = 'https://www.aihub.or.kr/'

# 웹 페이지 요청
response = requests.get(url)
response.raise_for_status()  # 요청이 성공했는지 확인

# BeautifulSoup 객체 생성
soup = BeautifulSoup(response.content, 'html.parser')

# 인기 데이터 TOP3 섹션 찾기
top3_section = soup.find('div', class_='secR')

# 각 데이터 항목 추출
data_list = top3_section.find_all('div', class_='list')

# 데이터 제목 추출
titles = []
for data in data_list:
    title = data.find('h3').get_text(strip=True)
    clean_title = title.split(']')[-1].strip()
    titles.append(clean_title)

# 추출한 데이터 출력
for idx, title in enumerate(titles, start=1):
    print(f"TOP {idx}: {title}")

TOP 1: 객체 간 관계성 인지용 한국형 비전 데이터
TOP 2: 손∙팔 협조에 의한 파지-조작 동작 데이터
TOP 3: 상용 자율주행차 야간 자동차 전용도로 데이터


In [9]:
import requests
from bs4 import BeautifulSoup

# 1. 웹 페이지 요청
url = 'https://ko.wikipedia.org/wiki/위키백과:대문'
response = requests.get(url)

# 2. 요청이 성공했는지 확인
if response.status_code == 200:
    # 3. BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.content, 'html.parser')

    # 4. 페이지 제목 추출
    title = soup.find('h1', id='firstHeading').text
    print(f"Title: {title}")

    # 5. 첫 번째 단락 추출
    first_paragraph = soup.find('p').text
    print(f"First paragraph: {first_paragraph}")
else:
    print(f"Failed to retrieve the web page. Status code: {response.status_code}")

Title: 위키백과:대문
First paragraph:  위키백과



In [10]:
!pip install selenium

     ---------------------------------------- 9.4/9.4 MB 10.1 MB/s eta 0:00:00
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
     ---------------------------------------- 58.8/58.8 kB ? eta 0:00:00
     -------------------------------------- 467.7/467.7 kB 9.7 MB/s eta 0:00:00
  Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 0.58.0
    Uninstalling websocket-client-0.58.0:
      Successfully uninstalled websocket-client-0.58.0
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: sniffio
    Found existing installation: sniffio 1.2.0
    Uninstalling sniffio-1.2.0:
      Successfully uninstalled sniffio-1.2.0
  Attempting uninstall: attrs


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
black 22.6.0 requires click>=8.0.0, but you have click 7.1.2 which is incompatible.


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# URL 설정
URL = "https://ko.wikipedia.org/wiki/위키백과:대문"

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--headless") # 브라우저 창을 띄우지 않음
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")

# 웹 드라이버 설정
driver = webdriver.Chrome(options=options)

try:
    # 위키백과 대문 페이지 열기
    driver.get(URL)

    # "우리 모두가 만들어가는 자유 백과사전"과 "문서 이하 내용" 추출
    main_content = driver.find_element(By.CSS_SELECTOR, "#mw-content-text > div.mw-content-ltr.mw-parser-output > div.main-box.main-top > div > p:nth-child(2)").text
    print("Main Content:", main_content)
finally:
    # 웹 드라이버 종료
    driver.quit()

Main Content: 우리 모두가 만들어가는 자유 백과사전
문서 674,592개와 최근 기여자 1,814명


In [14]:
# @title
!pip install scrapy

In [15]:
# @title
!scrapy startproject wikipedia_scraper

New Scrapy project 'wikipedia_scraper', using template directory 'C:\Users\Ys\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\Ys\workspace\python_kakao\kakaotech_aiden.ji\wikipedia_scraper

You can start your first spider with:
    cd wikipedia_scraper
    scrapy genspider example example.com


In [16]:
# @title
# wikipedia_scraper/spiders/wikipedia_spider.py
import scrapy

class WikipediaSpider(scrapy.Spider):
    name = "wikipedia"
    start_urls = [
        'https://ko.wikipedia.org/wiki/위키백과:대문',
    ]

    def parse(self, response):
        main_content = response.css('#mw-content-text > div.mw-content-ltr.mw-parser-output > div.main-pane > div.main-pane-right > div.wikipedia-ko.main-recommended.main-box').get()
        yield {
            'main_content': main_content,
        }

In [23]:
%%writefile -a wikipedia_scraper/wikipedia_scraper/settings.py
ROBOTSTXT_OBEY = False

Appending to wikipedia_scraper/wikipedia_scraper/settings.py


In [3]:
%%writefile run_scrapy.py
from scrapy.crawler import CrawlerProcess
from wikipedia_scraper.spiders.wikipedia_spider import WikipediaSpider

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'FEEDS': {
        'output.json': {
            'format': 'json',
            'encoding': 'utf8',
            'store_empty': False,
            'fields': None,
            'indent': 4,
        },
    },
})
process.crawl(WikipediaSpider)
process.start()

Overwriting run_scrapy.py


In [4]:
!python run_scrapy.py

Traceback (most recent call last):
  File "C:\Users\Ys\workspace\python_kakao\kakaotech_aiden.ji\day1\run_scrapy.py", line 2, in <module>
    from wikipedia_scraper.spiders.wikipedia_spider import WikipediaSpider
ModuleNotFoundError: No module named 'wikipedia_scraper.spiders'


In [2]:
import json
from pprint import pprint

with open('output.json','r') as f:
    data = json.load(f)
    pprint(data)

FileNotFoundError: [Errno 2] No such file or directory: 'output.json'